In this notebook, I will use linear chain CRF to predict aspect terms for the restaurants dataset, but this time without the terms themselves.

In [1]:
import pickle
with open('restaurant_iob.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [2]:
data[0]

[{'iob_tag': 'O',
  'text': 'But',
  'pos': 'CCONJ',
  'tag': 'CC',
  'dep': 'cc',
  'is_punct': False},
 {'iob_tag': 'O',
  'text': 'the',
  'pos': 'DET',
  'tag': 'DT',
  'dep': 'det',
  'is_punct': False,
  '-1: pos': 'CCONJ',
  '-1: tag': 'CC',
  '-1: dep': 'cc'},
 {'iob_tag': 'B',
  'text': 'staff',
  'pos': 'NOUN',
  'tag': 'NN',
  'dep': 'nsubj',
  'is_punct': False,
  '-1: pos': 'DET',
  '-1: tag': 'DT',
  '-1: dep': 'det',
  '-2: pos': 'CCONJ',
  '-2: tag': 'CC',
  '-2: dep': 'cc'},
 {'iob_tag': 'O',
  'text': 'was',
  'pos': 'VERB',
  'tag': 'VBD',
  'dep': 'ROOT',
  'is_punct': False,
  '-1: pos': 'NOUN',
  '-1: tag': 'NN',
  '-1: dep': 'nsubj',
  '-2: pos': 'DET',
  '-2: tag': 'DT',
  '-2: dep': 'det',
  '-3: pos': 'CCONJ',
  '-3: tag': 'CC',
  '-3: dep': 'cc'},
 {'iob_tag': 'O',
  'text': 'so',
  'pos': 'ADV',
  'tag': 'RB',
  'dep': 'advmod',
  'is_punct': False,
  '-1: pos': 'VERB',
  '-1: tag': 'VBD',
  '-1: dep': 'ROOT',
  '-2: pos': 'NOUN',
  '-2: tag': 'NN',
  '-2: d

In [3]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# Train/test split

80/20 split

In [4]:
import copy
train = data[0:2435]
test = data[2435:]

In [5]:
train[0]

[{'iob_tag': 'O',
  'text': 'But',
  'pos': 'CCONJ',
  'tag': 'CC',
  'dep': 'cc',
  'is_punct': False},
 {'iob_tag': 'O',
  'text': 'the',
  'pos': 'DET',
  'tag': 'DT',
  'dep': 'det',
  'is_punct': False,
  '-1: pos': 'CCONJ',
  '-1: tag': 'CC',
  '-1: dep': 'cc'},
 {'iob_tag': 'B',
  'text': 'staff',
  'pos': 'NOUN',
  'tag': 'NN',
  'dep': 'nsubj',
  'is_punct': False,
  '-1: pos': 'DET',
  '-1: tag': 'DT',
  '-1: dep': 'det',
  '-2: pos': 'CCONJ',
  '-2: tag': 'CC',
  '-2: dep': 'cc'},
 {'iob_tag': 'O',
  'text': 'was',
  'pos': 'VERB',
  'tag': 'VBD',
  'dep': 'ROOT',
  'is_punct': False,
  '-1: pos': 'NOUN',
  '-1: tag': 'NN',
  '-1: dep': 'nsubj',
  '-2: pos': 'DET',
  '-2: tag': 'DT',
  '-2: dep': 'det',
  '-3: pos': 'CCONJ',
  '-3: tag': 'CC',
  '-3: dep': 'cc'},
 {'iob_tag': 'O',
  'text': 'so',
  'pos': 'ADV',
  'tag': 'RB',
  'dep': 'advmod',
  'is_punct': False,
  '-1: pos': 'VERB',
  '-1: tag': 'VBD',
  '-1: dep': 'ROOT',
  '-2: pos': 'NOUN',
  '-2: tag': 'NN',
  '-2: d

Separate into X and y - the dict of features, and the correct IOB tag

In [6]:
y_train = [[token.pop('iob_tag') for token in sentence] for sentence in train]

In [7]:
[[token.pop('text') for token in sentence] for sentence in train]
X_train = train

# Fit the CRF Model

In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=500, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

c:\users\will\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=500,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

# Evaluate the Model

In [9]:
y_test = [[token.pop('iob_tag') for token in sentence] for sentence in test]
X_test = test

We remove the 'O' class otherwise our overall F1 score would be excessively high.

In [10]:
labels = list(crf.classes_)
labels.remove('O')

In [11]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.5772470904926429

In [12]:
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           B      0.671     0.534     0.595       738
           I      0.552     0.512     0.531       281

   micro avg      0.634     0.528     0.576      1019
   macro avg      0.611     0.523     0.563      1019
weighted avg      0.638     0.528     0.577      1019



The new model has an overall F1 score of 0.58.  It's not as good as the previous score of 0.77, but it's still a useful baseling.

Let's take a look at the raw predictions again.

In [13]:
for x_sentence, y_tag, y_prediction in zip(X_test, y_test, y_pred):
    for x_token, y_truth, y_pred_val in zip(x_sentence, y_tag, y_prediction):
        print(x_token['text'] + "\t\t\t tag: " + y_truth + "\t prediction: " + y_pred_val + " " + str(y_pred_val==y_truth))

I			 tag: O	 prediction: O True
was			 tag: O	 prediction: O True
here			 tag: O	 prediction: O True
a			 tag: O	 prediction: O True
few			 tag: O	 prediction: O True
weeks			 tag: O	 prediction: O True
back			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
had			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
worst			 tag: O	 prediction: O True
customer			 tag: B	 prediction: B True
service			 tag: I	 prediction: I True
experience			 tag: O	 prediction: I False
at			 tag: O	 prediction: O True
a			 tag: O	 prediction: O True
restaurant			 tag: O	 prediction: O True
ever			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
(			 tag: O	 prediction: O True
I			 tag: O	 prediction: O True
've			 tag: O	 prediction: O True
ordered			 tag: O	 prediction: O True
from			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
2nd			 tag: O	 prediction: O True
ave			 tag: O	 prediction: O True
.			 tag: O	 prediction: O

here			 tag: O	 prediction: O True
including			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
sculptured			 tag: O	 prediction: O True
iron			 tag: O	 prediction: B False
statue			 tag: O	 prediction: I False
on			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
table			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
Average			 tag: O	 prediction: O True
to			 tag: O	 prediction: O True
good			 tag: O	 prediction: O True
Thai			 tag: B	 prediction: O False
food			 tag: I	 prediction: O False
,			 tag: O	 prediction: O True
but			 tag: O	 prediction: O True
terrible			 tag: O	 prediction: O True
delivery			 tag: B	 prediction: B True
.			 tag: O	 prediction: O True
The			 tag: O	 prediction: O True
space			 tag: B	 prediction: B True
is			 tag: O	 prediction: O True
nice			 tag: O	 prediction: O True
but			 tag: O	 prediction: O True
when			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
order			 tag: O	 prediction: O True
our			 tag:

.			 tag: O	 prediction: O True
I			 tag: O	 prediction: O True
recommend			 tag: O	 prediction: O True
that			 tag: O	 prediction: O True
you			 tag: O	 prediction: O True
try			 tag: O	 prediction: O True
this			 tag: O	 prediction: O True
hidden			 tag: O	 prediction: B False
spot			 tag: O	 prediction: I False
which			 tag: O	 prediction: O True
will			 tag: O	 prediction: O True
surely			 tag: O	 prediction: O True
blowup			 tag: O	 prediction: O True
very			 tag: O	 prediction: O True
soon			 tag: O	 prediction: O True
The			 tag: O	 prediction: O True
garlic			 tag: B	 prediction: B True
mashed			 tag: I	 prediction: I True
potatoes			 tag: I	 prediction: I True
are			 tag: O	 prediction: O True
hands			 tag: O	 prediction: O True
down			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
best			 tag: O	 prediction: O True
in			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
city			 tag: O	 prediction: O True
!			 tag: O	 prediction: O True
Food			 tag: B

n			 tag: O	 prediction: O True
-			 tag: O	 prediction: O True
tart			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
order			 tag: O	 prediction: O True
yourself			 tag: O	 prediction: O True
a			 tag: O	 prediction: O True
hong			 tag: B	 prediction: B True
-			 tag: I	 prediction: I True
kong			 tag: I	 prediction: I True
styled			 tag: I	 prediction: I True
milk			 tag: I	 prediction: I True
and			 tag: O	 prediction: I False
tea			 tag: B	 prediction: I False
with			 tag: I	 prediction: I True
tapioca			 tag: I	 prediction: I True
pearls			 tag: I	 prediction: I True
(			 tag: I	 prediction: O False
hot			 tag: I	 prediction: O False
)			 tag: I	 prediction: O False
.			 tag: O	 prediction: O True
Aside			 tag: O	 prediction: O True
from			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
Sea			 tag: B	 prediction: O False
Urchin			 tag: I	 prediction: O False
,			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
chef			 tag: B	 prediction: B

will			 tag: O	 prediction: O True
be			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
last			 tag: O	 prediction: O True
place			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
stop			 tag: O	 prediction: O True
as			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
head			 tag: O	 prediction: O True
out			 tag: O	 prediction: O True
of			 tag: O	 prediction: O True
town			 tag: O	 prediction: O True
on			 tag: O	 prediction: O True
our			 tag: O	 prediction: O True
next			 tag: O	 prediction: O True
trip			 tag: O	 prediction: O True
to			 tag: O	 prediction: O True
New			 tag: O	 prediction: O True
York			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
Service			 tag: B	 prediction: B True
was			 tag: O	 prediction: O True
very			 tag: O	 prediction: O True
prompt			 tag: O	 prediction: O True
but			 tag: O	 prediction: O True
slightly			 tag: O	 prediction: O True
rushed			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True

who			 tag: O	 prediction: O True
wants			 tag: O	 prediction: O True
delicious			 tag: O	 prediction: O True
top			 tag: O	 prediction: O True
grade			 tag: O	 prediction: B False
caviar			 tag: B	 prediction: I False
and			 tag: O	 prediction: O True
fantastic			 tag: O	 prediction: O True
service			 tag: B	 prediction: B True
.			 tag: O	 prediction: O True
You			 tag: O	 prediction: O True
ca			 tag: O	 prediction: O True
n't			 tag: O	 prediction: O True
say			 tag: O	 prediction: O True
its			 tag: O	 prediction: O True
cheap			 tag: O	 prediction: O True
because			 tag: O	 prediction: O True
food			 tag: B	 prediction: O False
is			 tag: O	 prediction: O True
cheaper			 tag: O	 prediction: O True
in			 tag: O	 prediction: O True
Chinatown			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
I			 tag: O	 prediction: O True
have			 tag: O	 prediction: O True
eaten			 tag: O	 prediction: O True
there			 tag: O	 prediction: O True
3			 tag: O	 prediction: O True
-			 tag: O

steak			 tag: B	 prediction: B True
were			 tag: O	 prediction: O True
seasoned			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
cooked			 tag: O	 prediction: O True
to			 tag: O	 prediction: O True
perfection			 tag: O	 prediction: O True
,			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
lamb			 tag: B	 prediction: B True
sandwhich			 tag: I	 prediction: I True
is			 tag: O	 prediction: O True
great			 tag: O	 prediction: O True
for			 tag: O	 prediction: O True
heartier			 tag: O	 prediction: O True
appetites			 tag: O	 prediction: B False
.			 tag: O	 prediction: O True
Faan			 tag: O	 prediction: O True
is			 tag: O	 prediction: O True
sooo			 tag: O	 prediction: O True
good			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
We			 tag: O	 prediction: O True
saw			 tag: O	 prediction: O True
them			 tag: O	 prediction: O True
heating			 tag: O	 prediction: O True
up			 tag: O	 prediction: O True
at			 tag: O

,			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
works			 tag: O	 prediction: B False
.			 tag: O	 prediction: O True
I			 tag: O	 prediction: O True
went			 tag: O	 prediction: O True
with			 tag: O	 prediction: O True
5			 tag: O	 prediction: O True
friends			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
lingered			 tag: O	 prediction: O True
at			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
table			 tag: B	 prediction: O False
for			 tag: O	 prediction: O True
a			 tag: O	 prediction: O True
bit			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
did			 tag: O	 prediction: O True
n't			 tag: O	 prediction: O True
feel			 tag: O	 prediction: O True
rushed			 tag: O	 prediction: O True
at			 tag: O	 prediction: O True
all			 tag: O	 prediction: O True
even			 tag: O	 prediction: O True
though			 tag: O	 prediction: O True
there			 tag: O	 prediction: O True
was			 tag: O	 prediction: O True


chicken			 tag: I	 prediction: B False
.			 tag: O	 prediction: O True
Because			 tag: O	 prediction: O True
of			 tag: O	 prediction: O True
the			 tag: O	 prediction: O True
delicate			 tag: O	 prediction: O True
thin			 tag: O	 prediction: O True
crust			 tag: B	 prediction: O False
,			 tag: O	 prediction: O True
take			 tag: B	 prediction: O False
-			 tag: I	 prediction: O False
out			 tag: I	 prediction: O False
pies			 tag: I	 prediction: B False
get			 tag: O	 prediction: O True
soggy			 tag: O	 prediction: O True
in			 tag: O	 prediction: O True
their			 tag: O	 prediction: O True
boxes			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
I			 tag: O	 prediction: O True
suspect			 tag: O	 prediction: O True
this			 tag: O	 prediction: O True
restaurant			 tag: O	 prediction: O True
is			 tag: O	 prediction: O True
what			 tag: O	 prediction: O True
Rao			 tag: O	 prediction: O True
's			 tag: O	 prediction: O True
would			 tag: O	 prediction: O True
be			 tag: O	 pre

many			 tag: O	 prediction: O True
years			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
have			 tag: O	 prediction: O True
dined			 tag: O	 prediction: O True
at			 tag: O	 prediction: O True
many			 tag: O	 prediction: O True
restaraunts			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
When			 tag: O	 prediction: O True
you			 tag: O	 prediction: O True
enter			 tag: O	 prediction: O True
,			 tag: O	 prediction: O True
you			 tag: O	 prediction: O True
feel			 tag: O	 prediction: O True
like			 tag: O	 prediction: O True
you			 tag: O	 prediction: O True
have			 tag: O	 prediction: O True
entered			 tag: O	 prediction: O True
your			 tag: O	 prediction: O True
mom			 tag: O	 prediction: O True
's			 tag: O	 prediction: O True
kitchen			 tag: O	 prediction: O True
.			 tag: O	 prediction: O True
we			 tag: O	 prediction: O True
were			 tag: O	 prediction: O True
tired			 tag: O	 prediction: O True
and			 tag: O	 prediction: O True
cold			 tag: O	 predicti

Next: which features were most important in making our predictions?

In [14]:
from collections import Counter

def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(15))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-15:])

Top positive:
4.515974 O        tag:.
3.587735 O        pos:PRON
2.810508 O        pos:DET
2.729803 B        tag:``
2.638645 O        dep:aux
2.289212 O        -1: dep:quantmod
2.243440 O        tag:JJS
2.206088 O        -1: tag:VBZ
2.191445 O        is_punct
2.164092 O        pos:CCONJ
2.164092 O        tag:CC
2.144207 I        -2: tag:JJS
2.134981 O        dep:auxpass
2.066429 I        tag:-RRB-
2.021199 B        -3: dep:csubjpass

Top negative:
-1.182835 B        -1: dep:aux
-1.191195 I        pos:ADV
-1.193424 B        dep:det
-1.247642 B        pos:DET
-1.278350 B        -1: dep:mark
-1.329944 O        tag:VBN
-1.414571 B        -1: dep:case
-1.517487 I        -1: tag:-RRB-
-1.585555 B        is_punct
-1.716770 I        dep:poss
-1.724424 I        dep:amod
-1.901046 B        -2: dep:subtok
-1.930212 O        tag:-RRB-
-2.047419 I        pos:VERB
-2.559037 B        dep:prep


OK, this is an improvement on last time.  We can see that the features surrounding the token are useful in predicting its tag.